In [1]:
# common packages
import pandas as pd
import time

# # DL framework
import torch
from torchtext import data

from attractivedata import AttractiveData
from trainer import AttractiveTrainer

In [2]:
# torch.manual_seed(123)
# torch.cuda.manual_seed(123)
# torch.backends.cudnn.deterministic = True

## Load and prepare data

In [3]:
train_file = 'data/train.csv'
test_file = 'data/test.csv'
pretrained_file = 'glove.840B.300d'
config = {
    'max_seq': 40,
    'min_freq': 0,
    'batch_size': 64,
    'pretrained_file': pretrained_file
}


In [4]:
AttractiveData = AttractiveData(train_file, test_file, pretrained_file, config)

In [5]:
# for i, sentence in enumerate(AttractiveData.test_data):
#     if i == 3:
#         print(vars(AttractiveData.train_data[i]), vars(sentence))

In [6]:
len(AttractiveData.CATEGORIES_LABEL.vocab.freqs)

18

In [7]:
max_len = 0
a = AttractiveData.train_data
for i in range(len(a)):
    if len(a[i].Headline) >= max_len:
        max_len = len(a[i].Headline)
max_len

38

## Start to train

In [8]:
config['timestr'] = time.strftime("%Y%m%d-%H%M%S")
config['save_name'] = 'LSTM'
config['input_dim'] = len(AttractiveData.TEXT.vocab)
config['embedding_dim'] = 300
config['category_dim'] = len(AttractiveData.CATEGORIES_LABEL.vocab)
config['category_embedding_dim'] = 16
config['hidden_dim'] = 30
config['output_dim'] = 1
config['log_steps'] = 10
config['epochs'] = 500
config['lr'] = {
    'encoder': 1e-5,
    'embedding': 1e-5,
    'linear': 1e-4
}
config['num_layers'] = 1
config['kernel_size'] = 3
config['dropout'] = 0.0
config['train_len'] = AttractiveData.train_len
config['test_len'] = AttractiveData.test_len

pretrained_embeddings = AttractiveData.TEXT.vocab.vectors
print(pretrained_embeddings.shape)

torch.Size([12699, 300])


In [9]:
AttractiveTrainer = AttractiveTrainer(config, AttractiveData.device, AttractiveData.trainloader, pretrained_embeddings)

In [10]:
AttractiveTrainer.model, AttractiveTrainer.config['total_params'], AttractiveTrainer.config['total_learned_params']

(AttractiveNet(
   (embedding): AttractiveEmbedding(
     (token): TokenEmbedding(12699, 300, padding_idx=1)
   )
   (encoder): LSTM(300, 30, batch_first=True, bidirectional=True)
   (linear): Sequential(
     (0): Linear(in_features=120, out_features=30, bias=True)
     (1): ReLU()
     (2): Linear(in_features=30, out_features=1, bias=True)
   )
 ),
 3893041,
 83341)

In [11]:
AttractiveTrainer.train()

025566 |
====

EP_train | avg_loss: 0.43333857133984566 |
====
Epoch:  49%|████▉     | 247/500 [00:46<00:49,  5.11it/s]
EP_train | avg_loss: 0.433232088573277 |
====

EP_train | avg_loss: 0.43254614993929863 |
====
Epoch:  50%|████▉     | 249/500 [00:46<00:48,  5.23it/s]
EP_train | avg_loss: 0.4319168431684375 |
====

EP_train | avg_loss: 0.43174767680466175 |
====
Epoch:  50%|█████     | 251/500 [00:46<00:46,  5.30it/s]
EP_train | avg_loss: 0.431738612242043 |
====

EP_train | avg_loss: 0.43180958554148674 |
====
Epoch:  51%|█████     | 253/500 [00:47<00:45,  5.38it/s]
EP_train | avg_loss: 0.4318444160744548 |
====

EP_train | avg_loss: 0.4316370524466038 |
====
Epoch:  51%|█████     | 255/500 [00:47<00:45,  5.44it/s]
EP_train | avg_loss: 0.43132496904581785 |
====

EP_train | avg_loss: 0.43112758733332157 |
====
Epoch:  51%|█████▏    | 257/500 [00:48<00:44,  5.46it/s]
EP_train | avg_loss: 0.4307607617229223 |
====

EP_train | avg_loss: 0.43077997397631407 |
====
Epoch:  52%|█████▏   

## for classification, not better

## Below is testing

In [12]:
from attractivenet import AttractiveNet
PATH = './model/LSTM_20201101-150648_0.4040.500'
# load_model = TransformerModel(config).to(AttractiveData.device)
load_model = AttractiveNet(config).to(AttractiveData.device)
load_model.load_state_dict(torch.load(PATH))
load_model.eval()

AttractiveNet(
  (embedding): AttractiveEmbedding(
    (token): TokenEmbedding(12699, 300, padding_idx=1)
  )
  (encoder): LSTM(300, 30, batch_first=True, bidirectional=True)
  (linear): Sequential(
    (0): Linear(in_features=120, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=1, bias=True)
  )
)

In [30]:
def predict_attractive(sentence, category):
    # if len(sentence) < config['max_seq']:
    #     sentence += ['0'] * (config['max_seq'] - len(sentence))
    # else:
    #     sentence = sentence[:config['max_seq']]

    indexed_sentence = [AttractiveData.TEXT.vocab.stoi[t] for t in sentence]
    indexed_category = [AttractiveData.CATEGORIES_LABEL.vocab.stoi[category]]
    tensor_sentence = torch.LongTensor(indexed_sentence).to(AttractiveData.device)
    tensor_category = torch.LongTensor(indexed_category).to(AttractiveData.device)

    tensor_sentence = tensor_sentence.unsqueeze(0)
    # print(tensor_sentence.shape)

    prediction = load_model(tensor_sentence, tensor_category, phase='test')
    
    return prediction

In [23]:
# train mean = 3.15, test mean = 2.8
predict_list = []
with torch.no_grad():
    for i, sentence in enumerate(AttractiveData.test_data):
        prediction = predict_attractive(sentence.Headline, sentence.Category)
        predict_list.append(prediction.item())
        # predict_list.append(prediction.item())
AttractiveData.df_test['Label'] = predict_list
AttractiveData.df_test[['ID', 'Label']].to_csv(config['save_name'] + '.csv', index=False)

In [24]:
# train_category = list(AttractiveData.CATEGORIES_LABEL.vocab.freqs)
# test_category = list(AttractiveData.df_test['Category'].value_counts().keys())
# for each_test in test_category:
#     if each_test not in train_category:
#         print(each_test)
# print()
# for each_train in train_category:
#     if each_train not in test_category:
#         print(each_train)

## Below just for fun guess

In [25]:
import statistics
from sklearn.metrics import mean_squared_error

In [31]:
# train mean = 3.15, test mean = 2.8
train_list = []
for i, sentence in enumerate(AttractiveData.train_data):
    prediction = predict_attractive(sentence.Headline, sentence.Category)
    train_list.append(prediction.item())
    # train_list.append(prediction.item())
# print(train_list)
mean_squared_error(pd.read_csv('data/train.csv').sort_values(['ID']).Label.to_list(), train_list), statistics.mean(train_list), statistics.stdev(train_list)

(0.4204954766649157, 3.080096314935123, 0.31569246314417815)

In [27]:
train_list[0:5], pd.read_csv('data/train.csv').sort_values(['ID']).Label.to_list()[0:5]

([2.8401317596435547,
  2.3060123920440674,
  2.4998257160186768,
  2.7534849643707275,
  2.464578628540039],
 [4.0, 2.333333333333333, 4.5, 3.333333333333333, 4.0])

In [19]:
a = AttractiveData.df_train['Label'].to_list()
statistics.mean(a), statistics.stdev(a)

(0.0004084967320261136, 0.7295015193216009)

In [28]:
statistics.mean(predict_list), statistics.stdev(predict_list)

(2.6104562981013157, 0.31303651422594664)

In [29]:
baseline_list = pd.read_csv('baseline.csv').sort_values(['ID']).Label.to_list()
mean_squared_error(baseline_list, predict_list), statistics.mean(baseline_list), statistics.stdev(baseline_list)

(0.06358937625250528, 2.7156126782757597, 0.29355123275379763)

In [14]:
# LSTM my best
# statistics.mean(predict_list), statistics.stdev(predict_list)

(2.8167915543795683, 0.14611407210842048)

In [21]:
a = pd.read_csv('LSTM_base.csv').Label.to_list()
mean_squared_error(baseline_list, a), statistics.mean(a), statistics.stdev(a)

(0.1347375515605904, 2.8379913731293533, 0.1903582104725371)

In [14]:
AttractiveData.TEXT.vocab.itos[2]

'the'